In [17]:
using Pkg
Pkg.activate("C:/Users/lisah/Documents/Repos/ma-code")

  Activating project at `C:\Users\lisah\Documents\Repos\ma-code`


In [18]:
using DifferentialEquations, CSV, DataFrames, ForwardDiff, Distributions, LinearAlgebra

In [19]:
# functions i need from other file

abstract type AbstractHyperprm end

struct Hyperprm <: AbstractHyperprm
    w0::Float64
    n0::Float64
    a::Float64
    m::Float64
    M::Int64
    noise::Float64
end

function klausmeier!(du,u,p,t)
    du[1] = -u[1] - u[1] * u[2]^2 + p[1] # water compartment
    du[2] = u[1] * u[2]^2 - p[2] * u[2] # biomass compartment
end

function sol_klausmeier(hprm::Hyperprm)
    u0 = [hprm.w0; hprm.n0]
    p = [hprm.a; hprm.m]
    tspan = (0.0, hprm.M-1)

    prob = ODEProblem(klausmeier!, u0, tspan, p)
    sol = solve(prob,
        saveat=1.0  # consider specific time points
    )

    return DataFrame(time=sol.t, w=sol[1, :], n=sol[2, :])
end

function create_grid()
    a_vals = 0.0:0.1:2.0
    n_vals = 0.0:0.1:4.0

    grid = [(a, n) for n in n_vals, a in a_vals] 
    return grid
end

function randomize_data(df::DataFrame, noise::Float64)
    if noise == 0.0
        return df
    else
        df[!, "w"] .= df[!, "w"] .+ rand(Normal(0, noise), nrow(df))
        df[!, "n"] .= df[!, "n"] .+ rand(Normal(0, noise), nrow(df))
        return df
    end
end

randomize_data (generic function with 1 method)

In [20]:
function store_fish_data(w0::Float64,m::Float64,M::Int64,noise::Float64,df::DataFrame)
    CSV.write("c:/Users/lisah/Documents/Repos/ma-code/data/fisher/m0.45/fish_$(w0)_$(m)_$(M)_$(noise).csv", df)
end

store_fish_data (generic function with 1 method)

In [21]:
function compute_ll(x, hprm::Hyperprm, true_val::DataFrame)
    a, n0 = x
    hprm = Hyperprm(hprm.w0, hprm.n0, hprm.a, hprm.m, hprm.M, hprm.noise)
    pred_val = sol_klausmeier(hprm)
    ll = -0.5 * sum((true_val[:,"n"] - pred_val[:,"n"]) .^2) - 0.5 * sum((true_val[:,"w"] - pred_val[:,"w"]) .^2) # add up ll for both trajectories
    #ll = -0.5 * sum((true_val[:,"n"] - pred_val[:,"n"]) .^2)
    return ll
end

compute_ll (generic function with 2 methods)

In [22]:
function compute_fish(hprm) # returns trace of fisher information matrix at point pt hprm.n0, hprm.a

    
    return tr(FIM)
end

compute_fish (generic function with 1 method)

In [23]:
function gen_all_fish_data(M_vals, noise_vals)
    for M in M_vals
        for noise in noise_vals

            grid = create_grid()
            fish = zeros(41, 21)

            # evaluate fisher info on grid
            for i in range(1, 41)
                for j in range(1, 21)

                    pt = grid[i,j]
                    hprm = Hyperprm(w0, pt[2], pt[1], m, M, noise) #w0,n0,a,m,M

                    sol_true = sol_klausmeier(hprm) # returns df
                    sol_true = randomize_data(sol_true, hprm.noise) # include noise

                    x = [hprm.a, hprm.n0]
                    H = ForwardDiff.hessian(x -> compute_ll(x, hprm, sol_true), x)
                    FIM = -H
                    fish_val = tr(FIM)
                    fish[i,j] = fish_val
                end
            end
            
            #create data frame
            a_eval_pts = string.(0.0:0.1:2.0)
            df_fish = DataFrame(fish, a_eval_pts)

            store_fish_data(w0, m, M, noise, df_fish)
        end
    end
end

gen_all_fish_data (generic function with 1 method)

In [24]:
a_vals = [0.1, 0.9, 1.1, 1.7] 
n0_vals = [0.4, 0.5, 1.0, 1.3, 2.3]

index_combos = [[1,1], [2,4], [3,5], [4,2], [4,3]]

M_vals = [1000]
noise_vals = [0.0]
m = 0.45 # fix for now as given by literature
w0 = 0.95 # mean(a_vals)

gen_all_fish_data(M_vals, noise_vals)